In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from train_utils import TrainWrap
import pandas as pd


model_ckpt = 'roberta-base'

tw = TrainWrap(model_ckpt)


/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [4]:
defs = pd.read_csv('data/definitions.csv')

defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid', 
                    'definition': 'bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other', 
                    'definition': 'the use of coded or indirect language such as sarcasm, metaphor and circumlocution to disparage a protected group or individual, or to convey prejudicial and harmful views about them'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Hate Speech', 'name': 'labels', 
                    'definition': ''
                   }, ignore_index=True)


defs = defs.append({'promptName': 'Dehumanization', 'name': 'dehumanize', 
                    'definition': 'the process of depriving a person or group of positive human qualities.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Violent language', 'name': 'violence', 
                    'definition': 'calling for the unlawful exercise of physical force or intimidation by the exhibition of such force on a large group of people.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Genocide', 'name': 'genocide', 
                    'definition': 'calling for the deliberate killing of a large group of people.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Respect', 'name': 'respect', 
                    'definition': 'a feeling of deep admiration for someone or something elicited by their abilities, qualities, or achievements.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Insult', 'name': 'insult', 
                    'definition': 'to speak to or treat with disrespect or scornful abuse.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Humiliation', 'name': 'humiliate', 
                    'definition': 'making (someone) feel ashamed and foolish by injuring their dignity and self-respect, especially publicly.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Superiority', 'name': 'status', 
                    'definition': 'higher in rank, status, or quality.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Attack', 'name': 'attack_defend', 
                    'definition': 'an aggressive and violent action against a person or place, and defense is defined as speak or write in favor of (an action or person); attempt to justify.'
                   }, ignore_index=True)

/tmp/ipykernel_388145/996258701.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid',
/tmp/ipykernel_388145/996258701.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other',
/tmp/ipykernel_388145/996258701.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Hate Speech', 'name': 'labels',
/tmp/ipykernel_388145/996258701.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Dehumanization', 'name': 'dehumanize',
/tmp/ipyke

In [5]:
train_implicit = pd.read_csv('data/train_full.tsv', delimiter='\t')
train_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.trn.csv")

val_implicit = pd.read_csv('data/validation_full.tsv', delimiter='\t')
val_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.dev.csv")

In [6]:
implicit_labels = ['white_grievance','incitement', 'inferiority', 'stereotypical', 'irony', 'threatening', 'other']

sbic_labels = ['offensiveYN', 'intentYN', 'whoTarget']

_, train_sbic_enc = tw.preprocess_prompt(train_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
_, train_implicit_enc = tw.preprocess_prompt(train_implicit[~train_implicit['implicit_class'].isna()], t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
_, val_sbic_enc = tw.preprocess_prompt(val_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
_, val_implicit_enc = tw.preprocess_prompt(val_implicit[~val_implicit['implicit_class'].isna()], t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')


100%|████████████████████████████████████████████████████████████| 277/277 [00:00<00:00, 16987.94it/s]


In [8]:
from collections import Counter
from ast import literal_eval

# _, train_implicit_enc = tw.preprocess_prompt(train_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
target_categories = list(set([x for y in train_sbic['targetCategory'] for x in literal_eval(y)]))

Counter([x for y in train_sbic['targetCategory'] for x in literal_eval(y)])

Counter({'social': 642,
         'culture': 2471,
         'gender': 3535,
         'disabled': 753,
         'race': 3883,
         'victim': 2027,
         'body': 441})

In [9]:

import html
import re



def process_alpaca(text, label, instruction=None):
#     instruction = defs
    
    out = {'instruction': [],
           'input': [],
           'output': []
          
          }
    
    for i in range(len(text)):
        
        outp = label[i]
            
        inp = text[i].strip()
        inp = re.sub(r'(u/.*? |@.*?( |$))', "<user> ", inp)
        inp = re.sub(r'\u2019', "'", inp)
        inp = re.sub(r'[“”]', '"', inp)
        inp = re.sub(r'\\u[a-f0-9]{4}', "'", inp)
        inp = re.sub(r'\(https://.*\)', "(URL)", inp)

        inp = re.sub(r'[\s]{2,}', " ", inp)
        inp = html.unescape(inp).encode('ascii', errors='ignore').decode()
        
        if type(instruction) is str:
            out['instruction'].append(instruction)
        elif type(instruction) is list:
            out['instruction'].append(instruction[i])
            
        out['input'].append(inp)
        out['output'].append(outp)
    return out

In [10]:

def openprompt_to_prompt(test_enc):
    instructions = []
    test_prompt = []
    
    output = []
    
    for i in range(len(test_enc)):
        ins = '{target} is defined as {target_def}'.format(target=test_enc[i].meta['target'],
                                                                         target_def=test_enc[i].meta['definition'])
        if test_enc[i].meta['target_groups'] is not None:
            ins += 'Target groups include {0}'.format(', '.join(test_enc[i].meta['target_groups']))
        instructions.append(ins)
        test_prompt.append('Post:{text} Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(
            target=test_enc[i].meta['target'], 
            target_def=test_enc[i].meta['definition'],
            text=test_enc[i].text_a))
        
        output.append(('Yes' if test_enc[i].label == 2 else ('Unclear' if test_enc[i].label == 1 else 'No')))
    return process_alpaca(test_prompt, output, instructions)

In [11]:
train_sbic_enc[-1], train_sbic.iloc[-1]

({
   "guid": "35503",
   "label": 2,
   "meta": {
     "definition": "targeting a person by implicating a group they belong to. Group implications are distinguished from individual-only attacks or insults that do not invoke power dynamics between groups.",
     "target": "Group implications",
     "target_groups": null
   },
   "text_a": "\ud83d\udea8BREAKING: illegal alien 5x deported on 7 felonies found not guilty in the murder of Kate Steinle who he shot with a gun\n\nJury full of communists just allowed a foreigner to execute an American citizen without repercussion\n\nThis is a glimpse into the future if we don\u2019t take drastic measures",
   "text_b": "",
   "tgt_text": null
 },
 Unnamed: 0                                                          35503
 post                    🚨BREAKING: illegal alien 5x deported on 7 felo...
 targetMinority                                             ["immigrants"]
 targetCategory                                                ["culture"]
 ta

In [12]:
Counter([x.label for x in train_sbic_enc])

Counter({2: 36678, 0: 38587, 1: 31007})

In [13]:
train_sbic_prompt = openprompt_to_prompt(train_sbic_enc)

In [14]:
from datasets import Dataset

train_sbic_prompt = Dataset.from_dict(train_sbic_prompt)

In [15]:
train_implicit_prompt = Dataset.from_dict(openprompt_to_prompt(train_implicit_enc))

In [16]:
train_implicit_prompt[0]

{'instruction': 'White Grievance is defined as frustration over a minority group’s perceived privilege and casting majority groups as the real victims of racism. This language is linked to extremist behavior and support for violence.',
 'input': 'Post:people are upset on the cow protection issue pl . save your core support base muslims do not vote for you Q: Is this post a violation of White Grievance? Please answer Yes, No, or Unclear. A:',
 'output': 'No'}

In [17]:
from datasets import interleave_datasets

interleave_datasets([train_sbic_prompt.shuffle(), train_implicit_prompt.shuffle()]).to_json('crosstask.json')

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

26987147

In [18]:
train_sbic_prompt.to_json('train_sbic.json')


Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

44204674

In [19]:
train_implicit_prompt.to_json('train_implicit.json')

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

14222837

In [ ]:
instruction = '{target} is defined as {target_def}. Is this post a violation of any of the above definitions? Please answer Yes/No, and which policies. If there is no violations, output N/A.'
        